<a href="https://colab.research.google.com/github/nawnie/EveryDream/blob/main/EveryDream_Tools_4_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Please read the documentation here before you start.

I suggest reading this doc before you connect to your runtime to avoid using credits or being charged while you figure it out.

[link to old readme, new one is a wip](doc/AUTO_CAPTION.md)

In [ ]:
#@markdown # Install Dependencies and connect your Gdrive
#@markdown This will take a couple minutes, be patient and watch the output for "DONE!"
from IPython.display import clear_output
import subprocess
from tqdm.notebook import tqdm

print("downloading the required repos to use")
#downloading repos

!git clone https://github.com/nawnie/EveryDream.git

#@markdown Creates /content/drive/MyDrive/everydreamlogs/ckpt
Mount_to_Gdrive = True #@param{type:"boolean"} 

if Mount_to_Gdrive:
  from google.colab import drive
  drive.mount('/content/drive')
# Set working directory

%cd EveryDream

!git clone https://github.com/salesforce/BLIP scripts/BLIP
!pip install -q git+https://github.com/huggingface/transformers 

clear_output()

print("DONE! now, installing dependcies, this may seem to freeze for a moment at the start do not worry")

packages = [

# install requirements
'pandas>=1.3.5',
'timm',
'fairscale==0.4.4',
'diffusers[torch]==0.14.0',
'timm',
'aiofiles',
'colorama',
'pynvml'
'tensorrt'
]

for package in tqdm(packages, desc='Installing packages', unit='package'):
    if isinstance(package, tuple):
        package_name, extra_index_url = package
        cmd = f"pip install -q {package_name} --extra-index-url {extra_index_url}"
    else:
        cmd = f"pip install -q {package}"
        
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

!pip install -q nltk #oddly this does not install correctly with above process
clear_output()

clear_output()
print("DONE! installing dependcies make sure we are using python 3.10.x")
!python --version

In [ ]:
#@title Upload your input images or video into the EveryDream/input folder
#@markdown Run the following cell to create an upload button, allowing you to upload your images directly to this folder. 
#@markdown * it is faster to simply right click the input folder in the file browser available on the left toolbar
from google.colab import files

uploaded = files.upload()

for name, data in uploaded.items():
    with open(f"input/{name}", "wb") as f:
        f.write(data)
    print(f"Uploaded file: {name}")

#Blip, Blip-COCO, T5-FLAN, and Git Captioner
_____________________________________

"Blip", "Blip-COCO", "T5-FLAN", and "Git Captioner" are four transformer-based models developed for image captioning.

* "Blip" is trained on a large and diverse dataset of image captions, and is known for its strong performance on a wide range of image captioning tasks and datasets.

* "Blip-COCO" is a variant of "Blip" that has been fine-tuned on the COCO dataset, a popular benchmark for image captioning. Fine-tuning on COCO can improve a model's performance specifically on this dataset, but may not generalize as well to other datasets or tasks.

 * The COCO dataset consists of more than 330,000 images with five captions each, and is known for its high-quality captions and diversity of image types. 


* "T5-FLAN" is similar to "Blip" in that it is trained on a large and diverse dataset of image captions. However, it uses a different architecture called the "FLAN" (Feature-wise Linear Attention) transformer. The FLAN transformer is designed to better capture long-range dependencies and improve the modeling of feature interactions in the encoder, which can lead to better performance on certain tasks.

* "Git Captioner" is a model that generates image captions using a combination of computer vision and natural language processing techniques. It leverages pre-trained models for object detection and recognition, and then generates captions using a transformer-based language model.

When choosing among these models, consider your specific task or dataset needs. If you require a model that can perform well across a wide range of image captioning tasks and datasets, "Blip" or "T5-FLAN" may be a good choice. If you require strong performance specifically on the COCO dataset, consider fine-tuning "Blip-COCO". If you prefer a model that leverages object detection and recognition in addition to language modeling, consider "Git Captioner".

In addition to image captioning, "T5-FLAN" and "Git Captioner" have also been applied to tasks such as image generation and visual question answering.



In [ ]:
#@title Blip2 uses alot of ram, this cell will clear our ram usage
#@markdown Run this cell before captioning and if your model crashes while loading.
import gc
gc.collect()

In [ ]:
#@title Dream Captioner
input_folder = "" #@param {type:"string"}
#@markdown * The location of images to be captoned
model_name = "Salesforce/blip2-opt-2.7b" #@param ["microsoft/git-large-r-textcaps", "microsoft/git-large-textcaps", "microsoft/git-base-textcaps", "Salesforce/blip2-opt-2.7b", "Salesforce/blip2-opt-2.7b-coco", "Salesforce/blip2-flan-t5-xl"] {allow-input: true}
#@markdown * Select a model from the drop down menu, the ones on this menu have been tested to run in this colab enviroment.
model_storage = "/content/drive/MyDrive/Blip" #@param {type:"string"}
#@markdown * Choose where to save these models on the Gdrive
token_limit = 12 #@param {type:"slider", min:10, max:38, step:1}
#@markdown * Token length
replace_subject = True #@param {type:"boolean"}
#@markdown * This will find the first noun and replace it with the name of the folder it was in
#@markdown * EXAMPLE: if i caption images in a folder named Peter Griffin the first noun (person place or thing) in that caption will be replaced by "Peter Griffin"
use_cpu = False #@param {type:"boolean"}
#@markdown * this probably should not be used with colab but its there to try without a gpu

rename = ""
if replace_subject:
  rename = "--replace_subject"

cpu = ""
if use_cpu:
  cpu = "--force_cpu"

!python caption.py \
$cpu \
$rename \
--model $model_name \
--data_root "$input_folder" \
--Blip_location "$model_storage" \
--max_new_tokens $token_limit

In [ ]:
#@title Extract Frames from video

#@markdown Here we will use the folder input_vid 
!mkdir output/vid 

!python /scripts/extract_video_frames.py \
--vid_dir input \
--out_dir output/vid \
--format png \
--interval 10

In [ ]:
#@title Move the images to the input folder for captioning
!cp -r output/vid input

# Laion Downloader

* --laion_dir: directory with laion parquet files, default is ./laion

* --search_text: csv of words with AND logic, ex \"photo,man,dog\"

* --out_dir: directory to download files to, ive defaulted this to inputs so they can be captioned 

* --log_dir: directory for logs, if ommitted will not log, logs may be large!

* --column:column to search for matches, defaults is 'TEXT', but you could use 'URL' if you wanted",

* --limit: max number of matching images to download, warning: may be slightly imprecise due to concurrency and http errors, defaults is 100

* --min_hw: min height AND width of image to download, default is 512
  
* --force: forces a full download of all images, even if no search is provided, USE CAUTION!

* --parquet_skip: skips the first n parquet files on disk, useful to resume
        
* --verbose: additional logging of URL and TEXT 
        
* --test: skips downloading, for checking filters, use with "--verbose"


In [ ]:
!python scripts/download_laion.py \
--laion_dir ./laion \
--search_text "photo,man,dog" \
#--out_dir input \
#--log_dir logs \
#--column TEXT \
#--limit 100 \
#--min_hw 512 \
#--force False \
#--parquet_skip 0 \
#--Verbose False \
#--test not \


Here we can take our now captioned images and replace generic terms with our subjects

* --find: will search for a word in this case man

* --replace: will replace our found word with in this case bob smith

* --append_only: this will allow us to add a tag at he end 

In [ ]:
!python scripts/filename_replace.py \
--img_dir output \
--find "man" \
--replace "bob smith"

Now we can chose to create text files based on our file names, this is usefull for images with very long discriptions or tag list, windows has a limit of 256 characters, and files will not transfer correctly to a windows program if they are longer, moving these files in a zip is fine however and causes no issues


In [ ]:
!python scripts/createtxtfromfilename.py

Compress our images 

In [ ]:
!python scripts/compress_img.py \
--img_dir output \
--out_dir output/compressed_images \
--max_mp 1.5 
#--overwrite False \
#--Quality 95  \
#--noresize False \
#--delete \

## Download your DataSet from EveryDream/output

If you're on a colab you can use the cell below to push your output to your Gdrive.

In [ ]:

!mkdir /content/drive/MyDrive/Auto_Data_sets
!cp -r output/ /content/drive/MyDrive/Auto_Data_sets

## If not on colab/gdrive, the following will zip up your files for extraction

You'll still need to use your runtime's own download feature to download the zip.

![output zip](https://github.com/victorchall/EveryDream/blob/main/demo/output_zip.png?raw=1)

In [ ]:
import patoolib
from google.colab import files
import os

Zip_Location = "/content/drive/MyDrive/output" #@param {type:"string"}
#@markdown * this is the location containing your captioned images

!mkdir output/zip
!zip -r output/zip/output.zip "$Zip_Location"

if os.path.exists(output/zip/output.zip):
  files.download(output/zip/output.zip)
  !rm output/zip/output.zip
else:
  print("Error: File not found at specified path.")
